<a href="https://colab.research.google.com/github/hyesunyun/huggingface-lab/blob/main/huggingface_finetuning_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning HuggingFace 🤗 Transformer Models

*Adapted from https://huggingface.co/docs/transformers/en/training, https://huggingface.co/docs/transformers/en/tasks/sequence_classification, and https://huggingface.co/docs/transformers/en/tasks/summarization#load-billsum-dataset*

In this lab, we will go over how to fine-tune a model on text classification (sentiment analysis) and summarization tasks.

Transformers provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique.

We will also go over how we can use HuggingFace's [Trainer](https://huggingface.co/docs/transformers/en/trainer) to complete training and evaluation loop for PyTorch models. You only need to pass it the necessary pieces for training (model, tokenizer, dataset, evaluation function, training hyperparameters, etc.), and the Trainer class takes care of the rest. This makes it easier to start training faster without manually writing your own training loop. But at the same time, Trainer is very customizable and offers a ton of training options so you can tailor it to your exact training needs.

Make sure you have the runtime to GPU. You can pick T4 GPU.

Run the following cell to verify your GPU setup. You should see information about the GPU available for your session.

In [ ]:
! nvidia-smi

Fri Jan 10 19:51:39 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Install Packages

This is only needed for Google Colab users.

In [ ]:
# Transformers installation
! pip install transformers[torch] datasets evaluate accelerate rouge_score
# Install dependencies
! pip install torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=99cc399a6ead1edeb3ef4cccd54444dc9dfd25192b5478f05b3e75322b99449f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

If you would like to upload your fine-tuned model to the HuggingFace Hub. Then, I encourage you to create an account and login.

In [ ]:
## Uncomment and run the code below if you would like to login to HuggingFace
# from huggingface_hub import notebook_login

## When prompted, enter your authemtication token to login
## You can generate a secret auth token on HuggingFace account
# notebook_login()

## Text Classification (Sentiment Analysis)

In this section, we will finetune [DistilBERT](https://huggingface.co/distilbert/distilbert-base-uncased) on the [IMDb dataset](https://huggingface.co/datasets/stanfordnlp/imdb) to determine whether a movie review is positive or negative.

### Load IMDB dataset

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")

print(f"Splits of IMDB dataset: {imdb.keys()}")

print(f"Number of training samples: {len(imdb['train'])}")
print(f"Number of testing samples: {len(imdb['test'])}")

print(f"Example data instance: {imdb['test'][0]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Splits of IMDB dataset: dict_keys(['train', 'test', 'unsupervised'])
Number of training samples: 25000
Number of testing samples: 25000
Example data instance: {'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply fo

There are two fields in this dataset:

*   `text`: the movie review text.
*   `label`: a value that is either 0 for a negative review or 1 for a positive review.

In addition to loading datasets that are on HuggingFace's Hub, you can also load local/remote custom data files using `load_dataset` from `datasets` library.

For example:

```python
from datasets import load_dataset

# for CSV file
# pass your CSV files as a list if you are several CSV files (ex: train.csv, val.csv, test.csv)
dataset = load_dataset("csv", data_files="my_file.csv")

# for JSON file
dataset = load_dataset("json", data_files="my_file.json")

# for remote JSON file via HTTP
base_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/"
dataset = load_dataset("json", data_files={"train": base_url + "train-v1.1.json", "validation": base_url + "dev-v1.1.json"}, field="data")
```

For more details, please refer to this documentation: https://huggingface.co/docs/datasets/v3.2.0/loading#load

In [ ]:
# We will just use a subset of the dataset so we don't have to wait too long for fine-tuning to finish.
from datasets import DatasetDict, Dataset
import random

# get the first 2500 rows for train and 500 for test
train_indices = random.sample(range(len(imdb['train'])), 2500)
test_indices = random.sample(range(len(imdb['test'])), 500)
imdb_train = imdb['train'].select(train_indices)
imdb_test = imdb['test'].select(test_indices)

# create a new dataset in DatasetDict object
imdb = DatasetDict({
    'train': Dataset.from_list(imdb_train),
    'test':  Dataset.from_list(imdb_test)
})

print(f"Number of training samples: {len(imdb['train'])}")
print(f"Number of testing samples: {len(imdb['test'])}")

Number of training samples: 2500
Number of testing samples: 500


### Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the text field:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Create a preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

To apply the preprocessing function over the entire dataset, use HuggingFace's Datasets map function. You can speed up map by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

**Note**: Data Collator is used to form a batch by using a list of dataset elements as input.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the accuracy metric (see the Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
# you could also load "precision" or "f1" as well for this task

Then create a function that passes your predictions and labels to `compute` to calculate the accuracy:

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your compute_metrics function is ready to go now, and you'll return to it when you setup your training.

### Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

You're ready to start training your model now! Load DistilBERT with [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/v4.47.1/en/model_doc/auto#transformers.AutoModelForSequenceClassification) along with the number of expected labels, and the label mappings:

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:


1.   Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. At the end of each epoch, the Trainer will evaluate the accuracy and save the training checkpoint. **If you want to push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model).**
2.   Pass the training arguments to Trainer along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
output_dir = "./imdb_distilbert_model/"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none", # you can use wandb, tensorboard, etc to log your train process
    push_to_hub=False, # set to True if you want to push this model to the Hub
)

# Trainer applies dynamic padding by default when you pass tokenizer to it.
# In this case, you don’t need to specify a data collator explicitly.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## Once training is completed, upload your model to the Hub with the push_to_hub() method
# trainer.push_to_hub()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.457900,0.308484,0.878000
2,0.201100,0.339832,0.888000


TrainOutput(global_step=314, training_loss=0.28754728083397935, metrics={'train_runtime': 261.8688, 'train_samples_per_second': 19.094, 'train_steps_per_second': 1.199, 'total_flos': 656935842080112.0, 'train_loss': 0.28754728083397935, 'epoch': 2.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

{'eval_loss': 0.30848416686058044,
 'eval_accuracy': 0.878,
 'eval_runtime': 7.3137,
 'eval_samples_per_second': 68.365,
 'eval_steps_per_second': 4.375,
 'epoch': 2.0}

In [ ]:
checkpoint_number = 314

### Inference

Now that you've finetuned a model, you can use it for reference!



In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu" # checks if gpu is available
pipeline_device = 0 if device == "cuda" else -1 # for determining if we want to load model in GPU or CPU

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

## Easiest way to do inference is to use pipeline()
from transformers import pipeline

# load the fine-tuned model from locally saved model weights
classifier = pipeline("sentiment-analysis", model=f"/content/imdb_distilbert_model/checkpoint-{checkpoint_number}", device=pipeline_device)
classifier(text)

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9780646562576294}]

In [ ]:
# Using AutoTokenizer and AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

checkpoint_dir = f"/content/imdb_distilbert_model/checkpoint-{checkpoint_number}"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir).to(device)

# Tokenize the text and return PyTorch tensors
inputs = tokenizer(text, return_tensors="pt").to(device)
with torch.no_grad():
  # Pass your inputs to the model and return the logits
  logits = model(**inputs).logits

# Get the class with the highest probability, and use the model's id2label mapping to convert it to a text label
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'

## Summarization

Summarization creates a shorter version of a document or an article that captures all the important information. Summarization can be:

*   Extractive: extract the most relevant information from a document.
*   Abstractive: generate new text that captures the most relevant information.

This section will show you how to:

1. Finetune [T5](https://huggingface.co/google-t5/t5-small) on the California state bill subset of the [BillSum](https://huggingface.co/datasets/billsum) dataset for abstractive summarization.
2. Use your finetuned model for inference.

### Load BillSum dataset

Start by loading the smaller California state bill subset of the BillSum dataset from the 🤗 Datasets library:

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

print(f"Number of samples: {len(billsum)}")

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Number of samples: 1237


Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/v3.2.0/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

print(f"Number of train samples: {len(billsum['train'])}")
print(f"Number of test samples: {len(billsum['test'])}")

Number of train samples: 989
Number of test samples: 248


In [ ]:
# take a look at an example
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThis act shall be known, and may be cited, as the Pool Safety Act.\nSEC. 2.\nThe Legislature finds and declares all of the following:\n(a) Swimming pools provide children and their families with a wonderful opportunity for recreation, exercise, and fun. Keeping children safe during this activity is supported by parents and guardians, safety advocates, health providers, insurance companies, and the swimming pool industry.\n(b) According to both the federal Centers for Disease Control and Prevention’s National Center for Injury Prevention and Control and the State Department of Public Health’s EpiCenter data, drowning is the leading cause of death for California children one to four years of age, inclusive.\n(c) Additional children suffer near-drowning incidents and survive, but many of those children suffer irreversible brain injuries, which can lead to lifelong learning deficiencies that impact not on

There are two fields that you'll want to use:

* `text`: the text of the bill which'll be the input to the model.
* `summary`: a condensed version of text which'll be the model target.

### Preprocess

The next step is to load a T5 tokenizer to process text and summary:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    """
    Preprocess function for the summarization dataset.

    Args:
        examples: dataset examples

    Returns:
        model_inputs: dictionary of model inputs
    """
    ##### ADD YOUR CODE BELOW #####

    # prefix the inputs (examples["text"]) with a prompt so T5 knows this is a summarization task.
    inputs = [prefix + doc for doc in examples["text"]]
    # tokenize the inputs using max_length=1024 and truncate
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    # tokenize the target (examples["summary"]) to max_length=128 and truncate
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

To apply the preprocessing function over the entire dataset, use Datasets `map` method. You can speed up the map function by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
##### ADD YOUR CODE BELOW #####
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### Evaluate

Including a metric during training is often helpful for evaluating your model's performance. For this task, load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric.

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to compute to calculate the ROUGE metric:

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # During training, some tokens (typically padding tokens) in the labels might be set to -100.
    # The purpose of this line is to replace those -100 values with the actual padding token ID used by the tokenizer.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # stem the text before computing ROUGE
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # calculate the length of each generated summary (padding tokens are not counted)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # calculate the average length of the generated summaries
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

### Train

Load T5 with AutoModelForSeq2SeqLM:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

##### ADD CODE HERE #####
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. At the end of each epoch, the Trainer will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call `train()` to finetune your model.

In [ ]:
output_dir = "./billsum_t5_model/"

##### ADD CODE HERE #####

# add the missing arguments for Seq2SeqTrainingArguments
# follow the example from sentiment analysis to help you
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=50,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    report_to="none", # you can use wandb, tensorboard, etc to log your train process
    push_to_hub=False, # set to True if you want to push this model to the Hub
)

# load Seq2SeqTrainer with the correct arguments
# follow example from sentiment analysis
# then call train()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# OPTIONAL
# trainer.push_to_hub()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.766800,2.855574,0.132600,0.039300,0.109500,0.109600,20.000000
2,2.996700,2.661336,0.142400,0.047900,0.115900,0.115800,20.000000
3,2.903500,2.625189,0.142400,0.047600,0.115500,0.115400,20.000000


TrainOutput(global_step=186, training_loss=3.147560816939159, metrics={'train_runtime': 214.8991, 'train_samples_per_second': 13.806, 'train_steps_per_second': 0.866, 'total_flos': 803118249934848.0, 'train_loss': 3.147560816939159, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.6251888275146484,
 'eval_rouge1': 0.1424,
 'eval_rouge2': 0.0476,
 'eval_rougeL': 0.1155,
 'eval_rougeLsum': 0.1154,
 'eval_gen_len': 20.0,
 'eval_runtime': 17.6075,
 'eval_samples_per_second': 14.085,
 'eval_steps_per_second': 0.909,
 'epoch': 3.0}

The best model should be saved in `/content/billsum_t5_model/checkpoint-186`

### Inference

Come up with some text you’d like to summarize. For T5, you need to prefix your input depending on the task you’re working on.

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [ ]:
##### ADD CODE HERE #####
# either use pipeline() or AutoModelForSeq2SeqLM to do inference
# make sure you load the correct checkpoint directory

from transformers import pipeline

checkpoint_dir = "/content/billsum_t5_model/checkpoint-186"
summarizer = pipeline("summarization", model=checkpoint_dir)
summarizer(text)

Device set to use cuda:0
Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs . it's the most aggressive action on tackling the climate crisis in American history . no one making under $400,000 per year will pay a penny more in taxes."}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
inputs = tokenizer(text, return_tensors="pt").input_ids

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

tokenizer.decode(outputs[0], skip_special_tokens=True)

"the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. it's the most aggressive action on tackling the climate crisis in American history. it'll ask the ultra-wealthy and corporations to pay their fair share."

## Appendix

### Trainer Basic Tutorial

https://huggingface.co/docs/transformers/en/training#train-with-pytorch-trainer

### Parameter-Efficient Fine-Tuning

Even fine-tuning LLMs can become resource intensive. Hence, some researchers have come up with parameter-efficient fine-tuning (PEFT) methods.

LoRa (low-rank adaptation) is one of the most well-known and popular PEFT method. It's a popular method for fine-tuning models in a parameter-efficient way, by only training a few adapters, keeping the existing model untouched. LoRa is available in the [PEFT library](https://huggingface.co/docs/peft/index) by Hugging Face, which also supports various other PEFT methods.

QLoRa is another PEFT method that is even more efficient. This method applies LoRa to a quantized model (like 8-bit or 4-bit model).

If "full fine-tuning" exceeds the GPU RAM, then these training methods can help.

### More Examples

For more notebookes with examples on fine-tuning, please refer to the following resources:

*   https://huggingface.co/docs/transformers/notebooks#pytorch-nlp
*   https://github.com/NielsRogge/Transformers-Tutorials

### Hyperparameter Search

`Trainer` supports four hyperparameter search backends currently: optuna, sigopt, raytune and wandb.

https://huggingface.co/docs/transformers/en/hpo_train